# PyTorch Lightning abstraction basics

Putting it all together with PL abstraction mechanics

Let's first load all the necessary params

In [2]:
import numpy as np
import os.path as osp
import pytorch_lightning as pl

# Ensures this Notebook's reproducibility
pl.seed_everything(42, workers=True)

step = km.CONFIG['timestep']
params = km.PARAMS[str(step)]['flattened']

Global seed set to 42


## Model and training logic

In [ ]:
x_feats = params['x_shape'][-1]
y_feats = params['y_shape'][-1]

In [ ]:
print(f'x number of features: {x_feats}')
print(f'y number of features: {y_feats}')

In [79]:
mlp = km.parallel.models.LitMLP(
    in_channels=x_feats,
    hidden_channels=100,
    out_channels=y_feats
)
mlp

LitMLP(
  (net): Sequential(
    (0): Normalize()
    (1): Linear(in_features=4128, out_features=100, bias=True)
    (2): SiLU()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): SiLU()
    (5): Linear(in_features=100, out_features=100, bias=True)
    (6): SiLU()
    (7): Linear(in_features=100, out_features=552, bias=True)
  )
)

## Dataset creation and data loading mechanics

* `batch_size` sets the number of element in a batch of data.
* `num_workers` sets the number of workers the DataLoader can spawn to handle data loading and Dataset batching.

In [85]:
datamodule = km.parallel.data.FlattenedDataModule(
    batch_size=256,
    num_workers=16
)

## Orchestrating the training

All the training instrumentation is done by an object call the Trainer. You can fix parameters such as `max_epochs`, the `accelerator` type and `device` logical number.

Notably interesting: 
* `callbacks` to handle in-betweens
* `gradient_clip_val` and `gradient_clip_algorithm` to setup the gradient clipping
* `logger` to interface with loss and metrics logging
* `resume_from_checkpoint` helps resuming a previously initiated training
* `amp_backend` to switch to Nvidia Apex framework for Automatic Mixed Precision support

In [ ]:
trainer = pl.Trainer(
    max_epochs=1,
    logger=pl.loggers.tensorboard.TensorBoardLogger(
        save_dir=km.LOGS_PATH,
        name='flattened_mlp_logs',
        log_graph=True
    ),
    deterministic=True,
    amp_backend='apex'
)

Training CPU is a one-line

In [86]:
trainer.fit(model=mlp, datamodule=datamodule)

# Never forget the test
trainer.test(model=mlp, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 488 K 
------------------------------------
488 K     Trainable params
0         Non-trainable params
488 K     Total params
1.955     Total estimated model params size (MB)


Epoch 0:  89%|████████▉ | 3393/3816 [12:54<01:36,  4.38it/s, loss=0.438, v_num=19, train_loss=0.311]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 3816/3816 [14:30<00:00,  4.38it/s, loss=0.438, v_num=19, train_loss=0.311, val_loss_step=0.342, val_loss_epoch=0.417]


Now let's go multi-GPU

In [ ]:
trainer.fit(
    accelerator='gpu',
    devices=[0, 1, 2, 3],
    model=mlp,
    datamodule=datamodule
)
trainer.test(model=mlp, datamodule=datamodule)